In [19]:
# Human's emotion recognition using thermal images
import numpy as np
import glob
import random
from scipy.sparse.linalg import svds

np.set_printoptions(threshold=np.inf)

dbDir ='D:/ĐHSP/NCKH/Thermal Database/' #Database directory
eDir = {
    "angry":"anger",
    "disgust":"disgust",
    "fear":"fear",
    "happy":"happy",
    "neutral":"neutral",
    "sadness":"sadness",
    "surprise":"surprise"
} #Name of directory of each Emotion

In [2]:
def getSVXFiles(fd):
    return glob.glob(fd + '/*.svx')

In [3]:
def readSVX(file, frame):
    f = open(file,'rb') #Open file to read
    
    f.seek(128) #Skip 128bits
    [CA,CB] = np.divide(np.frombuffer(f.read(8), dtype=np.dtype('<i4'), count=2), 2**16)
    #Array from buffer
    #Data type: int32 (4 bytes/each - 'i4') 
    #Byte-order:  Litte-endian ('<')
    
    f.seek(1104) #Move to first frame to read
    
    img3d = np.zeros((frame,240,320))
    
    for i in range (0,frame):
        img = np.frombuffer(f.read(153600),dtype=np.dtype('>i2'))
        #Array from buffer
        #Data type: int16 (2 bytes/each - 'i2') 
        #Byte-order:  Big-endian ('>')
        img = img.dot(CB) + CA

        img2d = img.reshape(240,320) #Reshape 1 Dimension to 2 Dimensions for each frame
        img3d[i] = img2d #Add frame to array of frames
        
        f.seek(2592,1) #Move to next frame to read

    img3d = img3d.transpose(1,2,0) #Transpose to 240x320x[frame]
    f.close()
    return img3d

In [4]:
#readSVX(dbDir+eDir["angry"]+"/anan1.svx",2)

In [5]:
###
#* Parameter: 
#*
#* -  fd: folder
def loadThermalGlasses(fd, frame):
    files = getSVXFiles(fd) #Get list SVX files in folder
    faces = []
    
    for file in files:
        data = readSVX(file,frame).transpose(2,0,1) #Transpose to [frame]x240x320
        
        for i in range(frame-21, frame, 8):
            faces += [data[i]]
            
    print('Total faces: ', len(faces))
    return faces

In [6]:
faces = loadThermalGlasses(dbDir + eDir["angry"],300)

Total faces:  42


In [7]:
def divideDataset(faces):
    #Divide Dataset into 2, 70% for training & 30% for testing
    train = []
    test = []
    dsLength = len(faces)
    limitTest = dsLength*30//100
    
    for face in faces:
        if len(test) < limitTest and random.randint(1,10) >= 7:
            test += [face]
        else: 
            train += [face]
    
    return [train,test]

In [8]:
[train,test] = divideDataset(faces)

print(len(train),len(test))

30 12


In [9]:
def meanface(faces):
    total = np.zeros(faces[0].shape)
    count = len(faces)
    
    total = sum(faces)
    
    return total / count

In [10]:
#meanface(train)

In [28]:
# TODO: Read about SVD function
def training(faces, trainedFiles, k):
    avg = meanface(faces)
    
    M = []
    for face in faces:
        W = face - avg
        M += [W.reshape(-1)]
    M = np.array(M).transpose()
    print(M.shape)
    [U,W,V] = svds(M, full_matrics=True)
    print(U.shape)
    print(W.shape)
    print(V.shape)
    print(U[0])

In [29]:
training(train,'abc',15)

(76800, 30)


TypeError: svds() got an unexpected keyword argument 'full_matrics'